In [1]:
import os
import pyspark
conf = pyspark.SparkConf()
# conf.set('spark.ui.proxyBase'
# , '/user/' + os.environ['JUPYTERHUB_USER'] + '/proxy/4041')
conf.set('spark.sql.repl.eagerEval.enabled', True)
conf.set('spark.driver.memory','4g')
sc = pyspark.SparkContext(conf=conf)
spark = pyspark.SQLContext.getOrCreate(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/04 23:31:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


/Users/poojaakki/opt/anaconda3/lib/python3.9/site-packages/pyspark/sql/context.py:157: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [37]:
import pandas as pd
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import StringType

schema= StructType(
      [StructField('title',StringType(),True),
      StructField('text',StringType(),True),
      StructField('subject',StringType(),True),
      StructField('date',StringType(),True)])

df0 = pd.read_csv('/Users/poojaakki/Desktop/projects/Fake-News-Detection-Data-Analysis/datasets/Kaggle/fake.csv')
df1 = pd.read_csv('/Users/poojaakki/Desktop/projects/Fake-News-Detection-Data-Analysis/datasets/Kaggle/true.csv')
df_fake = spark.createDataFrame(df0,schema=schema)
df_true = spark.createDataFrame(df1,schema=schema)

In [38]:
# df_fake.show(20)

In [39]:
# df_true.show(20)

In [40]:
# add a label to the data for fake news as 0 and true news as 1 and shuffle using rand
from pyspark.sql.functions import lit, rand

df= df_true.withColumn('flag', lit(1)).union(df_fake.withColumn('flag', lit(0))).orderBy(rand())

In [41]:
df.printSchema()

root
 |-- title: string (nullable = true)
 |-- text: string (nullable = true)
 |-- subject: string (nullable = true)
 |-- date: string (nullable = true)
 |-- flag: integer (nullable = false)



In [42]:
#types of news and highest number of news in order
from pyspark.sql.functions import col

df.groupby('subject').count().sort(col("count").desc())

22/12/04 23:37:20 WARN TaskSetManager: Stage 21 contains a task of very large size (4983 KiB). The maximum recommended task size is 1000 KiB.
22/12/04 23:37:20 WARN TaskSetManager: Stage 24 contains a task of very large size (4983 KiB). The maximum recommended task size is 1000 KiB.


subject,count
politicsNews,11272
worldnews,10145
News,9050
politics,6841
left-news,4459
Government News,1570
US_News,783
Middle-east,778


In [43]:
# text pre-processing

from pyspark.sql.functions import col, split, lower, regexp_replace, length

df = df.select('title',(lower(regexp_replace('text', "[^a-zA-Z\\s]", "")).alias('text')), 'subject', 'date', 'flag')

In [44]:
# tokenize the text

from pyspark.ml.feature import Tokenizer

tokenizer = Tokenizer(inputCol='text', outputCol='words_token')

df = tokenizer.transform(df).select('title','words_token','subject','date','flag')

In [45]:
# df.first()

In [46]:
# remove the stop words

# tokens=['paris', 'reuters', '', 'plans', 'to', 'buy', 'a', 'new', 'presidential', 'jet', 'for', 'france', 's', 'emmanuel', 'macron', 'could', 'be']

# clean_tokens = [ tok for tok in tokens if tok ]

# clean_tokens

In [48]:
#remove empty string from the frame
from pyspark.sql.functions import udf
import pyspark.sql.functions as f
from pyspark.sql.types import ArrayType

def space_removal(f):
    clean_tokens = []
    for tok in f:
        if tok:
            clean_tokens.append(tok)
    return clean_tokens

udf_space_removal = udf(space_removal, ArrayType(StringType()))

df = df.withColumn('words_token', udf_space_removal(f.col('words_token')))

In [49]:
# df.first()

In [50]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/poojaakki/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [51]:
# remove the stop words
from pyspark.ml.feature import StopWordsRemover

stop_words_remover = StopWordsRemover(inputCol='words_token', outputCol='text').setStopWords(stop)

In [52]:
from pyspark.ml import Pipeline

stopWordRemovalPipeline = Pipeline(stages=[stop_words_remover])
pipelineFitRemoveStopWords = stopWordRemovalPipeline.fit(df)

In [53]:
df = pipelineFitRemoveStopWords.transform(df) #added here

In [54]:
# df.first()

In [55]:
# stematizing the words
# from nltk.stem.snowball import SnowballStemmer

# stemmer = SnowballStemmer(language='english')
# stemmer_udf = udf(lambda tokens: [stemmer.stem(token) for token in tokens], ArrayType(StringType()))

# df = df.withColumn("words_stemmed", stemmer_udf("text")).select('title','words_token','subject','date','flag','words_stemmed')

In [14]:
# df.first()

In [56]:
# df = df.withColumnRenamed("words_stemmed","text")

In [57]:
# df

In [58]:
# most freq words in data

from pyspark.sql.functions import concat_ws

df_freq = df.withColumn("text", concat_ws(" ", df["text"]))

# df_freq.show(5)

In [59]:
# most frequent fake words

df_fake_freq = df_freq.filter(df["flag"] == 0)

df_fake_text = df_fake_freq.select("text")


In [60]:
from pyspark.sql.functions import explode

df_fake_text = df_fake_text\
.withColumn("words", split(col("text"), " "))\
.where(length("text")>0)

df_freq_fake = df_fake_text\
.select(explode(col("words")).alias("word"))\
.groupBy("word").count()\

df_freq_fake.orderBy(col("count").desc()).show()

22/12/04 23:37:59 WARN TaskSetManager: Stage 27 contains a task of very large size (4983 KiB). The maximum recommended task size is 1000 KiB.


22/12/04 23:38:01 WARN TaskSetManager: Stage 28 contains a task of very large size (4983 KiB). The maximum recommended task size is 1000 KiB.


+---------+-----+
|     word|count|
+---------+-----+
|    trump|73933|
|     said|31013|
|   people|25963|
|president|25586|
|    would|23427|
|      one|22935|
|       us|22049|
|  clinton|18011|
|    obama|17813|
|     like|17621|
|   donald|17215|
|     also|15242|
|      new|14158|
|     news|14126|
|     even|13717|
|  hillary|13565|
|    white|12778|
|     time|12728|
|    state|12525|
|      via|11273|
+---------+-----+
only showing top 20 rows



In [124]:
#most true words

df_true_freq = df_freq.filter(df["flag"] == 1)

df_true_text = df_true_freq.select("text")

In [125]:
from pyspark.sql.functions import explode

df_true_text = df_true_text\
.withColumn("words", split(col("text"), " "))\
.where(length("text")>0)

df_freq_true = df_true_text\
.select(explode(col("words")).alias("word"))\
.groupBy("word").count()\

df_freq_true.orderBy(col("count").desc()).show()

22/12/04 23:09:54 WARN TaskSetManager: Stage 70 contains a task of very large size (4983 KiB). The maximum recommended task size is 1000 KiB.


22/12/04 23:10:06 WARN TaskSetManager: Stage 71 contains a task of very large size (4983 KiB). The maximum recommended task size is 1000 KiB.


+----------+-----+
|      word|count|
+----------+-----+
|      said|98985|
|     trump|54341|
|        us|41137|
|     state|36638|
|     would|31514|
|    presid|28367|
|    reuter|28306|
|republican|22108|
|    govern|20215|
|      year|19294|
|      hous|17525|
|       new|16786|
|      unit|16525|
|  democrat|16214|
|      also|15944|
|       say|15942|
|     senat|15705|
|     elect|15524|
|     peopl|15322|
|     parti|15002|
+----------+-----+
only showing top 20 rows



In [76]:
df_freq.printSchema()

root
 |-- title: string (nullable = true)
 |-- words_token: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- subject: string (nullable = true)
 |-- date: string (nullable = true)
 |-- flag: integer (nullable = false)
 |-- text: string (nullable = false)



In [125]:
df_freq.count()

22/12/04 22:10:01 WARN TaskSetManager: Stage 126 contains a task of very large size (4983 KiB). The maximum recommended task size is 1000 KiB.


44898

In [61]:
# drop unwanted coloumns

df_ml = df_freq.drop("title","words_token","date")
df_ml.printSchema()

root
 |-- subject: string (nullable = true)
 |-- flag: integer (nullable = false)
 |-- text: string (nullable = false)



In [62]:
from pyspark.sql.functions import monotonically_increasing_id 

df_ml = df_ml.select("*").withColumn("id", monotonically_increasing_id())

df_ml.show(200)

22/12/04 23:38:20 WARN TaskSetManager: Stage 34 contains a task of very large size (4983 KiB). The maximum recommended task size is 1000 KiB.
22/12/04 23:38:20 WARN TaskSetManager: Stage 35 contains a task of very large size (4983 KiB). The maximum recommended task size is 1000 KiB.
+---------------+----+--------------------+---+
|        subject|flag|                text| id|
+---------------+----+--------------------+---+
|           News|   0|using pimp pedoph...|  0|
|           News|   0|donald trump appe...|  1|
|   politicsNews|   1|mexico city reute...|  2|
|           News|   0|debate french tel...|  3|
|           News|   0|injured killed te...|  4|
|      worldnews|   1|johannesburg reut...|  5|
|           News|   0|trump presidentia...|  6|
|       politics|   0|one thing preside...|  7|
|   politicsNews|   1|vatican city reut...|  8|
|   politicsNews|   1|charleston sc reu...|  9|
|      worldnews|   1|townsville austra...| 10|
|        US_News|   0|us army stryker a...| 

In [63]:
# x values and Y values

x = df_ml.select('text').rdd.flatMap(lambda x: x).collect()
y = df_ml.select('flag').rdd.flatMap(lambda x: x).collect()

22/12/04 23:38:35 WARN TaskSetManager: Stage 38 contains a task of very large size (4983 KiB). The maximum recommended task size is 1000 KiB.
22/12/04 23:38:35 WARN TaskSetManager: Stage 39 contains a task of very large size (4983 KiB). The maximum recommended task size is 1000 KiB.


22/12/04 23:38:38 WARN TaskSetManager: Stage 42 contains a task of very large size (4983 KiB). The maximum recommended task size is 1000 KiB.
22/12/04 23:38:38 WARN TaskSetManager: Stage 43 contains a task of very large size (4983 KiB). The maximum recommended task size is 1000 KiB.


In [64]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=.25)

In [65]:
# print(x_train[0])
# print(y_train[0])

In [66]:
#vectorize the text

from sklearn.feature_extraction.text import TfidfVectorizer

In [67]:
vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

In [68]:
xv_train

<33673x184287 sparse matrix of type '<class 'numpy.float64'>'
	with 5459366 stored elements in Compressed Sparse Row format>

In [69]:
xv_train.toarray()[0]

array([0., 0., 0., ..., 0., 0., 0.])

In [70]:
from sklearn.linear_model import LogisticRegression

In [71]:
LR = LogisticRegression()
LR.fit(xv_train, y_train)

LogisticRegression()

In [72]:
LR.score(xv_test, y_test)

0.9867260579064588

In [73]:
pred_LR = LR.predict(xv_test)

In [74]:
print(classification_report(y_test, pred_LR))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5827
           1       0.98      0.99      0.99      5398

    accuracy                           0.99     11225
   macro avg       0.99      0.99      0.99     11225
weighted avg       0.99      0.99      0.99     11225



In [33]:
import re, string
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) 
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)    
    return text

In [34]:
import pandas as pd
def output_lable(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "Not A Fake News"
    
def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordopt) 
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_LR = LR.predict(new_xv_test)
    print("*****")
    #print(pred_LR[0])
    print(new_xv_test)
    print(type(pred_LR[0]))
    # pred_DT = DT.predict(new_xv_test)
    # pred_GBC = GBC.predict(new_xv_test)
    # pred_RFC = RFC.predict(new_xv_test)

    return print("\n\nLR Prediction: {} \n".format(output_lable(pred_LR[0])))

In [36]:
news = str(input())
manual_testing(news)

 GENEVA (Reuters) - North Korea and the United States clashed at a U.N. forum on Tuesday over their military intentions towards one another, with Pyongyang s envoy declaring it would  never  put its nuclear deterrent on the negotiating table. Japan, well within reach of North Korea s missiles, said the world must maintain pressure on the reclusive country to rein in its nuclear and missile programs and now was not the time for a resumption of multi-party talks. North Korea has pursued its weapons programs in defiance of U.N. Security Council sanctions and ignored all calls, including from major ally China, to stop, prompting a bellicose exchange of rhetoric between the North and the United States. North Korea justifies its weapons programs, including its recent threat to fire missiles towards the U.S. Pacific territory of Guam, by pointing to perceived U.S. hostility, such as military exercises with South Korea this week. U.S. disarmament ambassador Robert Wood told a U.N.-sponsored Co

*****
  (0, 156170)	0.016954924286470552
  (0, 154512)	0.02650776478056512
  (0, 154238)	0.006298867340384674
  (0, 154038)	0.005267483548213173
  (0, 153904)	0.013792583774074454
  (0, 153479)	0.008114144293972194
  (0, 153440)	0.16313172634949624
  (0, 152166)	0.014089192053292149
  (0, 151596)	0.01202210086534217
  (0, 151266)	0.005284337654835528
  (0, 149959)	0.021188517777870883
  (0, 146805)	0.010488395975697225
  (0, 146359)	0.0320198674929311
  (0, 146271)	0.02421962962263452
  (0, 144894)	0.009028343666646878
  (0, 144775)	0.022655260477836264
  (0, 143325)	0.006303291467913902
  (0, 142461)	0.003876503086332489
  (0, 142344)	0.015866133040270944
  (0, 142078)	0.010722981818063854
  (0, 140910)	0.0069399840275102325
  (0, 140553)	0.023524795941624228
  (0, 140261)	0.6292223730623426
  (0, 139476)	0.013601268636630753
  (0, 138924)	0.023799035284195698
  :	:
  (0, 28858)	0.008801759294083104
  (0, 27638)	0.011522671561104982
  (0, 27178)	0.01766483188099952
  (0, 26795)	0.0226